# Model



In [1]:
# Importing the necessary libraries
import os
from keras.preprocessing import image
import matplotlib.pyplot as plt 
import numpy as np
from keras.utils.np_utils import to_categorical
import random,shutil
from keras.models import Sequential
from keras.layers import Dropout,Conv2D,Flatten,Dense, MaxPooling2D, BatchNormalization
from keras.models import load_model

In [2]:
# Defining the generator function to return batches of data(images)
def generator(dir, gen=image.ImageDataGenerator(rescale=1./255), shuffle=True,batch_size=1,target_size=(24,24),class_mode='categorical' ):

    return gen.flow_from_directory(dir,batch_size=batch_size,shuffle=shuffle,color_mode='grayscale',class_mode=class_mode,target_size=target_size)

In [3]:
# Defining relevant variables    (this is just a sample dataset of a few images for demonstration purpose)
BS= 32          # Batch size
TS=(24,24)      # Target size
train_batch= generator('YawnDS/train',shuffle=True, batch_size=BS,target_size=TS)
valid_batch= generator('YawnDS/valid',shuffle=True, batch_size=BS,target_size=TS)
SPE= len(train_batch.classes)//BS   # Number of steps per epoch
VS = len(valid_batch.classes)//BS   # Validation steps
print(SPE,VS)

Found 2467 images belonging to 4 classes.
Found 433 images belonging to 4 classes.
77 13


In [4]:
# Defining model architecture
model = Sequential([ 
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(24,24,1)), # 2D Convolutional layer, 32 3x3 filters
    # relu was chosen to provide faster convergence during training and to effectively capture relevant features from data
    Conv2D(32,(3,3),activation='relu'),  # 2D Convolutional layer with 32 3x3 filters, relu activation function
    Conv2D(64, (3, 3), activation='relu'), # 2D Convolutional layer, 64 3x3 filters, relu
    
    Dropout(0.25), # Randomly turn off 25% neurons to prevent overfitting
    Flatten(), # Flatten output to a 1D array (since we only want classification)
    Dense(128, activation='relu'), # Fully connected layer to learn complex relationships and get relevant data
    Dropout(0.5), # One more dropout layer to further prevent overfitting
#output a softmax to squash the matrix into output probabilities
    Dense(4, activation='softmax') # Final layer to produce classification probabilities
    # Here, replace 4 with 2 if you aren't using the yawn eye dataset (in case it throws a Graph execution error)
])

In [5]:
# # Compile, Train and save the model    (Uncomment this cell to train and save the model)
# model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
# model.fit(train_batch, validation_data=valid_batch,epochs=15,steps_per_epoch=SPE ,validation_steps=VS)
# model.save('models/ddsCnn.h5', overwrite=True)

# Main code

In [6]:
# Importing relevant libraries
import cv2
import os
from keras.models import load_model
import numpy as np
from pygame import mixer
import time

pygame 2.2.0 (SDL 2.0.22, Python 3.11.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [7]:
# Alarm sound and initialising cascade classifiers
mixer.init()
sound = mixer.Sound('alarm.wav')

face = cv2.CascadeClassifier('haar cascade files\haarcascade_frontalface_alt.xml')
leye = cv2.CascadeClassifier('haar cascade files\haarcascade_lefteye_2splits.xml')
reye = cv2.CascadeClassifier('haar cascade files\haarcascade_righteye_2splits.xml')

In [8]:
# Labelling, loading model and creating frame (gui frame)
lbl=['Close','Open']

model = load_model('models/ddsCnn.h5')
path = os.getcwd()
cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_COMPLEX_SMALL
count=0
score=0
thicc=2
rpred=[99]
lpred=[99]

In [9]:
# Main loop to record video input, preprocess image, feed it to the model and give output
while(True):
    ret, frame = cap.read()
    height,width = frame.shape[:2] 

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = face.detectMultiScale(gray,minNeighbors=5,scaleFactor=1.1,minSize=(25,25))
    left_eye = leye.detectMultiScale(gray)
    right_eye =  reye.detectMultiScale(gray)

    cv2.rectangle(frame, (0,height-50) , (200,height) , (0,0,0) , thickness=cv2.FILLED )

    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y) , (x+w,y+h) , (100,100,100) , 1 )

    for (x,y,w,h) in right_eye:
        r_eye=frame[y:y+h,x:x+w]
        count=count+1
        r_eye = cv2.cvtColor(r_eye,cv2.COLOR_BGR2GRAY)
        r_eye = cv2.resize(r_eye,(24,24))
        r_eye= r_eye/255
        r_eye=  r_eye.reshape(24,24,-1)
        r_eye = np.expand_dims(r_eye,axis=0)
        rpred = model.predict_classes(r_eye)
        if(rpred[0]==1):
            lbl='Open' 
        if(rpred[0]==0):
            lbl='Closed'
        break

    for (x,y,w,h) in left_eye:
        l_eye=frame[y:y+h,x:x+w]
        count=count+1
        l_eye = cv2.cvtColor(l_eye,cv2.COLOR_BGR2GRAY)  
        l_eye = cv2.resize(l_eye,(24,24))
        l_eye= l_eye/255
        l_eye=l_eye.reshape(24,24,-1)
        l_eye = np.expand_dims(l_eye,axis=0)
        lpred = model.predict_classes(l_eye)
        if(lpred[0]==1):
            lbl='NOT Drowsy'   
        if(lpred[0]==0):
            lbl='Drowsy'
        break

    if(rpred[0]==0 and lpred[0]==0):
        score=score+1
        cv2.putText(frame,"Drowsy",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    # if(rpred[0]==1 or lpred[0]==1):
    else:
        score=score-1
        cv2.putText(frame,"Not Drowsy",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    
        
    if(score<0):
        score=0   
    cv2.putText(frame,'Score:'+str(score),(100,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    if(score>15):
        #person is feeling sleepy so we beep the alarm
        cv2.imwrite(os.path.join(path,'image.jpg'),frame)
        try:
            sound.play()
            
        except:  # isplaying = False
            pass
        if(thicc<16):
            thicc= thicc+2
        else:
            thicc=thicc-2
            if(thicc<2):
                thicc=2
        cv2.rectangle(frame,(0,0),(width,height),(0,0,255),thicc) 
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

# GUI 

In [10]:
# Importing relevant libraries
import tkinter as tk
from tkinter import messagebox
import subprocess
import webbrowser

In [11]:
# Function to start the system when the button is pressed
def start_clicked():
    messagebox.showinfo("Hello", "Please wait while the system loads...")
    subprocess.call(["python", "drowsiness detection.py"])

In [12]:
# Function to exit the system when the button is pressed
def exit_clicked():
    window.destroy()

In [13]:
# Main window
window = tk.Tk()
window.title("Homepage")
window.geometry("700x500")  # Set window size to 700x500

''

In [14]:
# Create a label for "Welcome" text
welcome_label = tk.Label(
    window, text="Welcome to the Drowsiness Detection System", font=("Helvetica", 24)
)
welcome_label.pack(pady=20)

In [15]:
# Create a frame to center the buttons
button_frame = tk.Frame(window)
button_frame.pack()

In [16]:
# Buttons
start_button = tk.Button(
    button_frame, text="Start Monitoring", command=start_clicked, font=("Helvetica", 16)
)
start_button.pack(pady=20)

# Create an "Exit" button
exit_button = tk.Button(
    button_frame, text="Exit", command=exit_clicked, font=("Helvetica", 16)
)
exit_button.pack(pady=10)

In [17]:
# Set background and foreground colors
welcome_label.configure(bg="lightgray", fg="blue")
start_button.configure(bg="green", fg="white")
exit_button.configure(bg="red", fg="white")

In [18]:
# Run the Tkinter event loop
window.mainloop()